In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

### 좋은 매칭점 찾기

In [4]:
img1 = cv2.imread('./img/taekwonv1.jpg')
img2 = cv2.imread('./img/figures.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# ORB 검출
detector = cv2.ORB_create()
kp1, des1 = detector.detectAndCompute(gray1, None)
kp2, des2 = detector.detectAndCompute(gray2, None)

# BF 매칭
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(des1, des2)

# 개선 작업
matches = sorted(matches, key=lambda x: x.distance)  # 거리가 작은 순으로 정렬
min_dist, max_dist = matches[0].distance, matches[-1].distance

# 최소 거리보다 ratio만큼 더 긴 매칭된 결과만 보겠다
ratio = 0.2    # ratio 조절 해가며 keypoints matching 결과 조정 클수록 더 많이 본다
good_thresh = (max_dist - min_dist) * ratio + min_dist

# 잘된 매칭점
good_matches = [i for i in matches if i.distance < good_thresh]
# good_matches = []
# for i in matches:
#     if i.distance < good_thresh:
#         good_matches.append(i)
print('matches:%d/%d, min:%.2f, max:%.2f, threshold:%.2f'
      %(len(good_matches),len(matches),min_dist,max_dist,good_thresh))

# 매칭 선 그리기
res1 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None,
                      flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
res2 = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None,
                      flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

cv2.imshow('bad',res1)
cv2.imshow('good',res2)
cv2.waitKey(0)
cv2.destroyAllWindows()

matches:18/127, min:24.00, max:78.00, threshold:34.80


In [14]:
# knn 개선
img1 = cv2.imread('./img/taekwonv1.jpg')
img2 = cv2.imread('./img/figures.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# ORB 검출
detector = cv2.ORB_create()
kp1, des1 = detector.detectAndCompute(gray1, None)
kp2, des2 = detector.detectAndCompute(gray2, None)

# BF 매칭
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)
matches = matcher.knnMatch(des1, des2, 2)

# 개선
ratio = 0.7
good_matches = [first for first,second in matches 
                if first.distance < second.distance*ratio]
# good_matches = []
# for first, second in matches:
#     if first.distance < second.distance * ratio:
#         good_matches.append(first)
res1 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matches, None,
                       flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
res2 = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None,
                       flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

                       
cv2.imshow('bad', res1)
cv2.imshow('good', res2)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 매칭 영역 원근 변환

In [5]:
img1 = cv2.imread('./img/taekwonv1.jpg')
img2 = cv2.imread('./img/figures.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# ORB, BF-Hamming, knnMatch
detector = cv2.ORB_create()
kp1, desc1 = detector.detectAndCompute(gray1, None)
kp2, desc2 = detector.detectAndCompute(gray2, None)
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)
matches = matcher.knnMatch(desc1, desc2, 2)

# 이웃거리 비율
ratio = 0.75
good_matches = [first for first,second in matches
                if first.distance<second.distance*ratio]
print('good:%d/%d'%(len(good_matches), len(matches)))

src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])
dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches])


# 원근 변환 행렬 구하기
mtrx, mask = cv2.findHomography(src_pts, dst_pts)

h, w = img1.shape[:2]
pts = np.float32([ [[0,0]],[[0,h-1]],[[w-1,h-1]],[[w-1,0]] ]) # 각 꼭지점 좌표

dst = cv2.perspectiveTransform(pts,mtrx)
img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

res = cv2.drawMatches(img1,kp1,img2,kp2,good_matches,None,
                      flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

cv2.imshow('homography', res)
cv2.waitKey(0)
cv2.destroyAllWindows()


good:23/500


#### Video 읽기 및 다루기

In [30]:
video_path = './img/big_buck.avi'
cap = cv2.VideoCapture(video_path)

if cap.isOpened():
    while True:     # True 동안 다음 프레임 읽어오기
        ret, frame = cap.read()
        if not ret:  # 영상 끝!!
            break
        cv2.imshow('video', frame)
        if cv2.waitKey(1) & 0xff == 27:    # 25ms 지연
            break
else:
    print('no video')

cap.release()
cv2.destroyAllWindows()

## 배경 제거

In [37]:
cap = cv2.VideoCapture('./img/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS)   # 영상 프레임 수 구하기
delay = int(1000/fps)   # waitKey에 들어갈 숫자

# 배경 제거 객체 생성
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # background subtractor mask
    fgmask = fgbg.apply(frame)
    cv2.imshow('frame',frame)
    cv2.imshow('bgsubtract',fgmask)
    if cv2.waitKey(delay) & 0xff == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [35]:
cap = cv2.VideoCapture('./img/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)

fgbg = cv2.createBackgroundSubtractorMOG2()
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    fgmask = fgbg.apply(frame)
    cv2.imshow('frame', frame)
    cv2.imshow('bgsub', fgmask)
    if cv2.waitKey(delay) & 0xff == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [36]:
cap = cv2.VideoCapture('./img/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS)   # 영상 프레임 수 구하기
delay = int(1000/fps)   # waitKey에 들어갈 숫자

# 배경 제거 객체 생성
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
fgbg2 = cv2.createBackgroundSubtractorMOG2()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # background subtractor mask
    fgmask1 = fgbg.apply(frame)
    fgmask2 = fgbg2.apply(frame)
    cv2.imshow('frame',frame)
    cv2.imshow('MOG',fgmask1)
    cv2.imshow('MOG2',fgmask2)
    if cv2.waitKey(delay) & 0xff == 27:
        break
cap.release()
cv2.destroyAllWindows()

### Optical Flow

In [40]:
cap = cv2.VideoCapture('./img/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)

color = np.random.randint(0,255,(200,3))
lines = None
prevImg = None
# 반복 중지 조건
termcriteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    img_draw = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 맨 첫 프레임(영상의 시작)
    if prevImg is None:
        prevImg = gray
        # 추적선을 그릴 이미지를 프레임 크기랑 같게 생성
        lines = np.zeros_like(frame)
        # 추적 시작을 위한 Keypoint 검출
        prevPt = cv2.goodFeaturesToTrack(prevImg, 200, 0.01, 10)
    else:
        nextImg = gray
        nextPt, status, err = cv2.calcOpticalFlowPyrLK(prevImg, nextImg,
                                                       prevPt, None,
                                                       criteria=termcriteria)
        # previous, next 서로 대응점이 있는 keypoint
        prevMv = prevPt[status==1]
        nextMv = nextPt[status==1]
        for i, (p,n) in enumerate(zip(prevMv,nextMv)):
            px,py=p.ravel()
            nx,ny=n.ravel()
            px,py,nx,ny = int(px),int(py),int(nx),int(ny)
            # 이전, 다음 keypoint 연결 선
            cv2.line(lines,(px,py),(nx,ny), color[i].tolist(),2)
            # 새로운 keypoint에 점
            cv2.circle(img_draw, (nx,ny), 2, color[i].tolist(), -1)
        img_draw = cv2.add(img_draw, lines)
        prevImg = nextImg
        prevPt = nextMv.reshape(-1,1,2)
    cv2.imshow('OpticalFlow-LK', img_draw)
    key = cv2.waitKey(delay)
    if key == 27:    #esc
        break
    elif key == 8:   # backspace
        prevImg = None
cap.release()
cv2.destroyAllWindows()

In [3]:
# Flow의 결과 그리기 함수
def drawFlow(img, flow, step=16):
    h,w = img.shape[:2]
    idx_y,idx_x = np.mgrid[step/2:h:step, step/2:w:step].astype(np.int32)
    indices = np.stack((idx_x,idx_y), axis=-1).reshape(-1,2) #(x,y)
    
    for x,y in indices:
        # Grid의 Index위치 점 찍기
        cv2.circle(img, (x,y), 1, (0,255,0), -1)
        # Grid Index의 Flow 결과값 (이동 거리)
        dx, dy = flow[y,x].astype(np.int32)
        # (x,y), (dx,dy) 선
        cv2.line(img, (x,y), (x+dx,y+dy), (0,255,0), 2, cv2.LINE_AA)

prev = None
cap = cv2.VideoCapture('./img/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if prev is None:
        prev = gray
    else:
        flow = cv2.calcOpticalFlowFarneback(prev,gray,None,
                                            0.5,3,15,3,5,1.1,
                                            cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        drawFlow(frame, flow)
        prev = gray
    cv2.imshow('OpticalFlow-GF', frame)
    if cv2.waitKey(delay) & 0xff == 27:
        break
cap.release()
cv2.destroyAllWindows()

## Tracking API

In [2]:
trackers = [cv2.TrackerMIL_create, cv2.TrackerKCF_create, cv2.TrackerCSRT_create]
trackerIdx = 0
tracker = None
isFirst = True
video_src = './img/highway.mp4'

cap = cv2.VideoCapture(video_src)
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)
win_name = 'Tracking APIs'

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    img_draw = frame.copy()
    if tracker is None:
        cv2.putText(img_draw, 'Press the Space button', (100,80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2, cv2.LINE_AA)
    else: 
        ok, bbox = tracker.update(frame)
        (x,y,w,h) = bbox
        if ok:
            cv2.rectangle(img_draw, (int(x),int(y)),(int(x+w),int(y+h)),
                          (0,255,0), 2, 1)
        else:
            cv2.putText(img_draw, 'Tracking Fail!', (100,80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2, cv2.LINE_AA)
    trackerName = tracker.__class__.__name__
    cv2.putText(img_draw, str(trackerIdx)+':'+trackerName, (100,20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2, cv2.LINE_AA)
    cv2.imshow(win_name, img_draw)
    key = cv2.waitKey(delay) & 0xff # == 27 ← ord('esc')
    
    # 스페이스바, 숫자 0,1,2 비디오 재생 중 누르는 키보드 이벤트
    if key == ord(' ') or (video_src != 0 and isFirst):
        isFirst = False
        roi = cv2.selectROI(win_name, frame, False) # 초기 객체 위치
        if roi[2] and roi[3]:
            tracker = trackers[trackerIdx]()    # 트랙커 객체 생성
            isInit = tracker.init(frame, roi)

    elif key in range(48,51):
        trackerIdx = key-48  # key 48, 49, 50 → 0, 1, 2
        if bbox is not None:
            tracker = trackers[trackerIdx]()
            isInit = tracker.init(frame, bbox)

    elif key == 27:
        break

cap.release()
cv2.destroyAllWindows()
    

### HOG 디스크립터

In [6]:
# default 디텍터를 위한 객체 생성, 설정
hogdef = cv2.HOGDescriptor()
hogdef.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# dailer 디텍터 위한 객체 생성, 설정 / winsize, blocksize, bin(x축 나오는 갯수)
hogdaim = cv2.HOGDescriptor((48,96),(16,16),(8,8),(8,8),9)
hogdaim.setSVMDetector(cv2.HOGDescriptor_getDaimlerPeopleDetector())

cap = cv2.VideoCapture('./img/walking.avi')
mode = True
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break
    elif ret:
        if mode:
            # default로 검출
            found, _ = hogdef.detectMultiScale(img)
            for (x,y,w,h) in found:
                cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,255))
        else:
            # daimler로 검출
            found, _ = hogdaim.detectMultiScale(img)
            for (x,y,w,h) in found:
                cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0))
        cv2.putText(img, 'Detector:%s'%('Default'if mode else 'Diamler'),
                    (10,50), cv2.FONT_HERSHEY_DUPLEX,1,(0,255,0),1)
        cv2.imshow('walk', img)
        key = cv2.waitKey(1)
        if key == 27:
            break
        elif key == ord(' '):
            mode = not mode
cap.release()
cv2.destroyAllWindows()

In [ ]:
'./weights/파일명'

In [12]:
image = cv2.imread('./img/people.jpg')

image_resized = cv2.resize(image, (755,500))

cascade_face_detector = cv2.CascadeClassifier('./weights/haarcascade_frontalface_default.xml')
face_detections = cascade_face_detector.detectMultiScale(image_resize,
                                                        scaleFactor=2,
                                                        minNeighbors=4)
# [(x,y,w,h), (x,y,w,h), (x,y,w,h), (x,y,w,h), (x,y,w,h),(x,y,w,h)]

for (x,y,w,h) in face_detections:
    cv2.rectangle(image_resized, (x,y), (x+w,y+h), (0,255,0),2)

cv2.imshow('people', image_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
img = cv2.imread('./img/people.jpg')
img_resize = cv2.resize(img, (755,500))

cascade_eye_detector = cv2.CascadeClassifier('./weights/haarcascade_eye.xml')
eye_detections = cascade_eye_detector.detectMultiScale(img_resize, 
                                                      scaleFactor=1.05,
                                                      minNeighbors=6,
                                                      minSize=(10,10),
                                                      maxSize=(30,30))

for (x,y,w,h) in eye_detections:
    cv2.rectangle(img_resize, (x,y), (x+w,y+h), (0,255,255), 2)

cv2.imshow('eye', img_resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
img = cv2.imread('./img/people.jpg')
img_resize = cv2.resize(img, (755,500))

cascade_face_detector = cv2.CascadeClassifier('./weights/haarcascade_frontalface_default.xml')
cascade_eye_detector = cv2.CascadeClassifier('./weights/haarcascade_eye.xml')

face_detections = cascade_face_detector.detectMultiScale(img_resize,
                                                         scaleFactor=2)
eye_detections = cascade_eye_detector.detectMultiScale(img_resize, 
                                                      scaleFactor=1.05,
                                                      minNeighbors=6,
                                                      minSize=(10,10),
                                                      maxSize=(30,30))
for (x,y,w,h) in face_detections:
    cv2.rectangle(img_resize, (x,y), (x+w,y+h), (0,255,0), 2)

for (x,y,w,h) in eye_detections:
    cv2.rectangle(img_resize, (x,y), (x+w,y+h), (0,255,255), 2)

cv2.imshow('all', img_resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 얼굴 인식 LBPH

In [20]:
!pip install pillow

In [22]:
import os
import numpy as np
import cv2
from PIL import Image

In [25]:
train_image_path = './img/yalefaces/train'

all_train_img_name = os.listdir(train_image_path)
# ['subject01.centerlight.gif',
#  'subject01.glasses.gif',
#  'subject01.happy.gif',
#  'subject01.leftlight.gif',...]
all_train_img_path = []
for i in all_train_img_name:
    new_path = train_image_path + '/' + i
    all_train_img_path.append(new_path)
# ['./img/yalefaces/train/subject01.centerlight.gif',
#  './img/yalefaces/train/subject01.glasses.gif',
#  './img/yalefaces/train/subject01.happy.gif',
#  './img/yalefaces/train/subject01.leftlight.gif',...]

In [26]:
def fetch_preprocessed_image_data(image_paths):
    faces_of_image = []
    ids_of_image = []
    for image_path in image_paths:
        image = Image.open(image_path).convert('L')  # cv2.cvtColor(cv2.COLOR_BGR2GRAY)
        # to Grayscale
        image_np = np.array(image, 'uint8')
        image_file_name = os.path.split(image_path)[1]   # subject01.centerlight.gif
        id_of_image = int(image_file_name.split('.')[0].replace('subject',''))    #int(01) == 1
        ids_of_image.append(id_of_image)
        faces_of_image.append(image_np)
    return np.array(ids_of_image), faces_of_image

In [28]:
ids_of_train_image, faces_of_train_image = fetch_preprocessed_image_data(all_train_img_path)

In [37]:
from skimage.feature import local_binary_pattern
original_image = faces_of_train_image[6]
# LBPH 적용
lbp_image = local_binary_pattern(original_image, 8*3, 3)
# 정규화 0~255
lbp_image = ((lbp_image - lbp_image.min())/(lbp_image.max()-lbp_image.min())) * 255
# 다시 이미지로
lbp_image = lbp_image.astype(np.uint8)             

In [38]:
cv2.imshow('or',original_image)
cv2.imshow('xor',lbp_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [39]:
# 객체 생성
lbph_recognizer = cv2.face.LBPHFaceRecognizer_create(radius=5,neighbors=15,
                                                    grid_x=10,grid_y=10)
lbph_recognizer.train(faces_of_train_image, ids_of_train_image)
# lbph_recognizer.train(X,y)

In [40]:
test_image_path = './img/yalefaces/test'
# 위에서 진행한 train과 동일한 내용
all_test_image_file_name = os.listdir(test_image_path)
all_test_image_paths = [os.path.join(test_image_path,f) for f in all_test_image_file_name]
ids_of_test_image, faces_of_test_image = fetch_preprocessed_image_data(all_test_image_paths)

In [41]:
id_pred, confidence_pred = lbph_recognizer.predict(faces_of_test_image[0])
id_pred, confidence_pred

(2, 139.49445435935314)

In [42]:
ids_pred = []
# Prediction
for face_of_test_image in faces_of_test_image:
    # face_of_test_image ← 이미지에 대해 예측한 Y값 (몇번 사람인지)
    id_pred, _ = lbph_recognizer.predict(face_of_test_image)
    ids_pred.append(id_pred)

In [47]:
# [0] 숫자 바꿔가며 보시면 됩니다!!

cv2.putText(faces_of_test_image[1], f'Pred: {str(id_pred)}', (5, 30), cv2.FONT_HERSHEY_PLAIN, 1.1, (0, 255, 0))
cv2.putText(faces_of_test_image[1], f'True: {str(ids_of_test_image[0])}', (5, 60), cv2.FONT_HERSHEY_PLAIN, 1.1, (0,255,0))

cv2.imshow('test',faces_of_test_image[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [43]:
!pip install scikit-learn

In [44]:
from sklearn.metrics import accuracy_score

accuracy_score(ids_of_test_image, ids_pred)

0.8666666666666667